In [1]:
import pandas as pd
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import LSTM, Dense, Embedding, Dropout
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import pad_sequences
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
df = pd.read_csv('/content/drive/MyDrive/Election Project/Complete 2002.csv')
desire_col = ['constituency_number', 'province', 'candidate_name','candidate_party','candidate_votes','candidate_share','outcome']
data = df[desire_col]

In [3]:
df.isnull().sum()

election_date          0
election_type          0
contest_status         0
constituency_number    0
constituency_name      0
province               0
assembly               0
voter_reg              0
validated_votes        0
votes_disq             0
candidateID            0
candidate_name         0
candidate_party        0
candidate_votes        0
candidate_share        0
candidate_rank         0
outcome                0
dtype: int64

In [4]:
len(df['constituency_number'].unique())

272

In [5]:
df

,election_date,election_type,contest_status,constituency_number,constituency_name,province,assembly,voter_reg,validated_votes,votes_disq,candidateID,candidate_name,candidate_party,candidate_votes,candidate_share,candidate_rank,outcome
0,2002-10-10,General Election,Contested,NA-1,Peshawar 1,Khyber Paktunkhwa (NWFP),National,233907.0,65642.0,1552.0,cxEaOeoEpx,Shabir Ahmad,Muttahidda Majlis-e-Amal Pakistan,37179.0,0.566390,1.0,Win
1,2002-10-10,General Election,Contested,NA-1,Peshawar 1,Khyber Paktunkhwa (NWFP),National,233907.0,65642.0,1552.0,JXBry680U4,Usman Bashir Bilour,Awami National Party,23002.0,0.350416,2.0,Loss
2,2002-10-10,General Election,Contested,NA-1,Peshawar 1,Khyber Paktunkhwa (NWFP),National,233907.0,65642.0,1552.0,20bRcNfCcx,Sajid Abdullah,Pakistan Tehreek-e-Insaf,2029.0,0.030910,3.0,Loss
3,2002-10-10,General Election,Contested,NA-1,Peshawar 1,Khyber Paktunkhwa (NWFP),National,233907.0,65642.0,1552.0,3d2MkDSVO4,"M. Khurshid Khan, Adv.",Independents,1537.0,0.023415,4.0,Loss
4,2002-10-10,General Election,Contested,NA-1,Peshawar 1,Khyber Paktunkhwa (NWFP),National,233907.0,65642.0,1552.0,GZMMicWhS6,Muhammad Muazzam Butt,Pakistan Muslim League (Qaid-e-Azam),1417.0,0.021587,5.0,Loss
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2061,2002-11-17,Re-Polling,Contested,NA-262,Kill Abbdullah,Balochistan,National,265538.0,85585.0,3521.0,PTxyUS4GG2,Bismillah Khan,Pakistan Peoples Party Parliamentarians,14763.0,0.172495,3.0,Loss
2062,2002-11-17,Re-Polling,Contested,NA-262,Kill Abbdullah,Balochistan,National,265538.0,85585.0,3521.0,kSO5CsxLmr,Lala Jan,Pakistan Muslim League (Qaid-e-Azam),862.0,0.010072,4.0,Loss
2063,2002-11-17,Re-Polling,Contested,NA-262,Kill Abbdullah,Balochistan,National,265538.0,85585.0,3521.0,SGpLAs1AX6,Abdul Khaliq,Independents,453.0,0.005293,5.0,Loss
2064,2002-11-17,Re-Polling,Contested,NA-262,Kill Abbdullah,Balochistan,National,265538.0,85585.0,3521.0,jmLR89ucvW,Abdul Aziz,Independents,173.0,0.002021,6.0,Loss


In [6]:
max_candidate_name_length = 40
max_candidate_party_length = 40

In [7]:
def tokenize_and_pad(data, max_len):
    sequences = pad_sequences(data, maxlen=max_len, padding='post')
    return sequences.tolist()

In [8]:
data

,constituency_number,province,candidate_name,candidate_party,candidate_votes,candidate_share,outcome
0,NA-1,Khyber Paktunkhwa (NWFP),Shabir Ahmad,Muttahidda Majlis-e-Amal Pakistan,37179.0,0.566390,Win
1,NA-1,Khyber Paktunkhwa (NWFP),Usman Bashir Bilour,Awami National Party,23002.0,0.350416,Loss
2,NA-1,Khyber Paktunkhwa (NWFP),Sajid Abdullah,Pakistan Tehreek-e-Insaf,2029.0,0.030910,Loss
3,NA-1,Khyber Paktunkhwa (NWFP),"M. Khurshid Khan, Adv.",Independents,1537.0,0.023415,Loss
4,NA-1,Khyber Paktunkhwa (NWFP),Muhammad Muazzam Butt,Pakistan Muslim League (Qaid-e-Azam),1417.0,0.021587,Loss
...,...,...,...,...,...,...,...
2061,NA-262,Balochistan,Bismillah Khan,Pakistan Peoples Party Parliamentarians,14763.0,0.172495,Loss
2062,NA-262,Balochistan,Lala Jan,Pakistan Muslim League (Qaid-e-Azam),862.0,0.010072,Loss
2063,NA-262,Balochistan,Abdul Khaliq,Independents,453.0,0.005293,Loss
2064,NA-262,Balochistan,Abdul Aziz,Independents,173.0,0.002021,Loss


In [9]:
data['candidate_name'] = data.groupby('constituency_number')['candidate_name'].transform(lambda x: x.astype(str).tolist())
data['candidate_party'] = data.groupby('constituency_number')['candidate_party'].transform(lambda x: x.astype(str).tolist())

# Extract unique constituencies
constituencies = data['constituency_number'].unique()

<ipython-input-9-df51f43c78f4>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['candidate_name'] = data.groupby('constituency_number')['candidate_name'].transform(lambda x: x.astype(str).tolist())
<ipython-input-9-df51f43c78f4>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['candidate_party'] = data.groupby('constituency_number')['candidate_party'].transform(lambda x: x.astype(str).tolist())


In [10]:
constituencies

array(['NA-1', 'NA-10', 'NA-100', 'NA-101', 'NA-102', 'NA-103', 'NA-104',
       'NA-105', 'NA-106', 'NA-107', 'NA-108', 'NA-109', 'NA-11',
       'NA-110', 'NA-111', 'NA-112', 'NA-113', 'NA-114', 'NA-115',
       'NA-116', 'NA-117', 'NA-118', 'NA-119', 'NA-12', 'NA-120',
       'NA-121', 'NA-122', 'NA-123', 'NA-124', 'NA-125', 'NA-126',
       'NA-127', 'NA-128', 'NA-129', 'NA-13', 'NA-130', 'NA-131',
       'NA-132', 'NA-133', 'NA-134', 'NA-135', 'NA-136', 'NA-137',
       'NA-138', 'NA-139', 'NA-14', 'NA-140', 'NA-141', 'NA-142',
       'NA-143', 'NA-144', 'NA-145', 'NA-146', 'NA-147', 'NA-148',
       'NA-149', 'NA-15', 'NA-150', 'NA-151', 'NA-152', 'NA-153',
       'NA-154', 'NA-155', 'NA-156', 'NA-157', 'NA-158', 'NA-159',
       'NA-16', 'NA-160', 'NA-161', 'NA-162', 'NA-163', 'NA-164',
       'NA-165', 'NA-166', 'NA-167', 'NA-168', 'NA-169', 'NA-17',
       'NA-170', 'NA-171', 'NA-172', 'NA-173', 'NA-174', 'NA-175',
       'NA-176', 'NA-177', 'NA-178', 'NA-179', 'NA-18', 'NA-18

In [11]:
all_features, all_labels = [], []

for constituency_number in constituencies:
    group = data[data['constituency_number'] == constituency_number]

    features = group[['province', 'candidate_name', 'candidate_party', 'candidate_votes', 'candidate_share']]
    labels = group['outcome'].astype('category').cat.codes

    all_features.append(features)
    all_labels.append(labels)

processed_data = pd.concat(all_features, ignore_index=True)
processed_data['outcome'] = pd.concat(all_labels, ignore_index=True)

In [12]:
# Convert text columns to sequences
tok = Tokenizer(num_words=10000)
tok.fit_on_texts(processed_data['candidate_name'].astype(str))
processed_data['candidate_name_padded'] = tok.texts_to_sequences(processed_data['candidate_name'].astype(str))

tok.fit_on_texts(processed_data['candidate_party'].astype(str))
processed_data['candidate_party_padded'] = tok.texts_to_sequences(processed_data['candidate_party'].astype(str))

In [13]:
X_train, X_test, y_train, y_test = train_test_split(processed_data[['province', 'candidate_name_padded', 'candidate_party_padded', 'candidate_votes', 'candidate_share']], processed_data['outcome'], test_size=0.2, random_state=42)

In [14]:
from keras.layers import Bidirectional

# Model
model = Sequential()
model.add(Embedding(10000, 128, input_length=max_candidate_name_length))
model.add(LSTM(64, return_sequences=True))  # Return sequences to pass to the next LSTM layer
model.add(Dropout(0.2))
model.add(LSTM(32, return_sequences=True))  # Return sequences again
model.add(LSTM(32))  # No need to return sequences for the last LSTM layer
model.add(Dense(16, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


In [15]:
X_train_padded = pad_sequences(X_train['candidate_name_padded'].tolist(), maxlen=max_candidate_name_length, padding='post', truncating='post', dtype='float32')

# Assuming max_candidate_name_length is the length of your padded sequences
timesteps = max_candidate_name_length
features = 1  # Assuming each element in the sequence is a single feature

X_train_padded = X_train_padded.reshape((X_train_padded.shape[0], timesteps, features))

# Train the model
model.fit(X_train_padded, y_train, epochs=10, validation_split=0.2)
#model.save('/content/drive/MyDrive/Election Project/first_model.h5')
model.summary()

Epoch 1/10
42/42 [==============================] - 8s 85ms/step - loss: 0.4520 - accuracy: 0.8531 - val_loss: 0.3967 - val_accuracy: 0.8701
Epoch 2/10
42/42 [==============================] - 2s 46ms/step - loss: 0.3878 - accuracy: 0.8698 - val_loss: 0.3863 - val_accuracy: 0.8701
Epoch 3/10
42/42 [==============================] - 2s 46ms/step - loss: 0.3909 - accuracy: 0.8698 - val_loss: 0.3868 - val_accuracy: 0.8701
Epoch 4/10
42/42 [==============================] - 2s 49ms/step - loss: 0.3872 - accuracy: 0.8698 - val_loss: 0.3871 - val_accuracy: 0.8701
Epoch 5/10
42/42 [==============================] - 2s 46ms/step - loss: 0.3880 - accuracy: 0.8698 - val_loss: 0.3872 - val_accuracy: 0.8701
Epoch 6/10
42/42 [==============================] - 3s 73ms/step - loss: 0.3882 - accuracy: 0.8698 - val_loss: 0.3864 - val_accuracy: 0.8701
Epoch 7/10
42/42 [==============================] - 2s 45ms/step - loss: 0.3875 - accuracy: 0.8698 - val_loss: 0.3868 - val_accuracy: 0.8701
Epoch 8/10
42

In [16]:
X_test_padded = pad_sequences(X_test['candidate_name_padded'].tolist(), maxlen=max_candidate_name_length, padding='post', truncating='post', dtype='float32')
X_test_padded = X_test_padded.reshape((X_test_padded.shape[0], timesteps, features))

# Evaluate the model on the test set
evaluation = model.evaluate(X_test_padded, y_test)
print("Test Loss:", evaluation[0])
print("Test Accuracy:", evaluation[1])

13/13 [==============================] - 0s 11ms/step - loss: 0.4009 - accuracy: 0.8623
Test Loss: 0.4008893072605133
Test Accuracy: 0.8623188138008118


In [17]:
predictions_2002 = model.predict(X_test_padded)

13/13 [==============================] - 1s 11ms/step


In [18]:
predictions_2002

array([[0.14389619],
       [0.14389355],
       [0.14389141],
       [0.14388865],
       [0.14389293],
       [0.14389198],
       [0.14388648],
       [0.14388879],
       [0.14388968],
       [0.1438895 ],
       [0.14388938],
       [0.14388892],
       [0.14389773],
       [0.14389873],
       [0.14388895],
       [0.14388894],
       [0.14389193],
       [0.14388703],
       [0.1438883 ],
       [0.14390051],
       [0.143892  ],
       [0.14389142],
       [0.14388633],
       [0.14389096],
       [0.14389247],
       [0.1438887 ],
       [0.14389445],
       [0.14389178],
       [0.14388818],
       [0.14388897],
       [0.14388518],
       [0.14389317],
       [0.14389114],
       [0.14389122],
       [0.14389156],
       [0.14388686],
       [0.14389552],
       [0.14388816],
       [0.14389029],
       [0.14389573],
       [0.14388925],
       [0.14388947],
       [0.14389177],
       [0.1438919 ],
       [0.1438928 ],
       [0.14389224],
       [0.14388727],
       [0.143